# Python functies voor Odata3
Functies voor ophalen, inspecteren en samenvoegen van data van CBS

In [1]:
import pandas as pd
import requests

def get_odata(target_url):
    
    """"
    De functie gebruikt een API genaamd OData om data van het CBS op te halen.
    De data wordt in stukken opgehaald en in een pandas dataframe gezet.
    De URL moet er zo uitzien: "https://odata4.cbs.nl/CBS/83765NED"
    De code van de tabel die je zoekt vindt je via Statline.
    Ga naar de data in Statline op de website van het CBS en kijk naar de URL om de code te vinden.
    """
    
    data = pd.DataFrame()
    while target_url:
        r = requests.get(target_url).json()
        data = data.append(pd.DataFrame(r['value']))
        
        if '@odata.nextLink' in r:
            target_url = r['@odata.nextLink']
        else:
            target_url = None
            
    return data

In [2]:
def get_observations(table_url, url_filter = ""):
    
    if url_filter == "":
        target_url = table_url + "/Observations"
    elif "?$filter=" in url_filter:
        target_url = table_url + "/Observations" + url_filter
    else:
        print("WAARSCHUWING! FILTER NIET GOED GEFORMATTEERD. VERGELIJK MET VOORBEELD OF GA NAAR ")
        print("\n https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/filters-odata-v4")
        #return None
        pass
    
    data = get_odata(target_url)
    print(data.head)
    
    return(data)

lastyear_url = "https://opendata.cbs.nl/oDataAPI/OData/84583NED"

In [4]:
#test = get_odata(lastyear_url)
test['url'].values

array(['https://opendata.cbs.nl/oDataAPI/OData/84583NED/TableInfos',
       'https://opendata.cbs.nl/oDataAPI/OData/84583NED/UntypedDataSet',
       'https://opendata.cbs.nl/oDataAPI/OData/84583NED/TypedDataSet',
       'https://opendata.cbs.nl/oDataAPI/OData/84583NED/DataProperties',
       'https://opendata.cbs.nl/oDataAPI/OData/84583NED/CategoryGroups',
       'https://opendata.cbs.nl/oDataAPI/OData/84583NED/WijkenEnBuurten'],
      dtype=object)

In [9]:
test_info = get_odata("https://opendata.cbs.nl/oDataAPI/OData/84583NED/TableInfos")

In [13]:
print(test_info.columns)
test_info[['Description','ExplanatoryText','Frequency','ShortDescription','ShortTitle','Summary','Title']].values

Index(['Catalog', 'DefaultPresentation', 'DefaultSelection', 'Description',
       'ExplanatoryText', 'Frequency', 'GraphTypes', 'ID', 'Identifier',
       'Language', 'MetaDataModified', 'Modified', 'OutputStatus', 'Period',
       'ReasonDelivery', 'SearchPriority', 'ShortDescription', 'ShortTitle',
       'Source', 'Summary', 'Title'],
      dtype='object')


array([["INHOUDSOPGAVE\r\n\r\n1. Toelichting\r\n2. Definities en verklaring van symbolen\r\n3. Koppelingen naar relevante tabellen en artikelen\r\n4. Bronnen en methoden\r\n5. Meer informatie \r\n\r\n\r\n1. TOELICHTING\r\n\r\nOverzicht van statistische gegevens van gemeenten, wijken en buurten in Nederland. \r\n\r\nGegevens beschikbaar over: 2019.\r\n\r\nStatus van de cijfers:\r\nDefinitief, tenzij in de toelichting bij het onderwerp expliciet is vermeld dat het voorlopige cijfers betreft.\r\n\r\nWijzigingen per mei 2020\r\nAlle variabelen binnen het thema bedrijfsvestigingen zijn toegevoegd. Binnen het thema voorzieningen zijn de variabelen afstand tot huisartsenpraktijk, afstand tot grote supermarkt en afstand tot kinderdagverblijf toegevoegd.\r\n\r\nWijzigingen per 23 oktober 2019\r\nDe omgevingsadressendichtheid en bijbehorende mate van stedelijkheid zijn gecorrigeerd op gemeente- en  wijkniveau. In de eerdere berekening waren een aantal adressen weggevallen, wat leidde tot een ver

In [14]:
test_wijken=get_odata("https://opendata.cbs.nl/oDataAPI/OData/84583NED/WijkenEnBuurten")
test_wijken.columns

Index(['Description', 'DetailRegionCode', 'Key', 'Municipality', 'Title'], dtype='object')

In [22]:
test_data = get_odata("https://opendata.cbs.nl/oDataAPI/OData/84583NED/DataProperties")
print(test_data.columns)

Index(['Datatype', 'Decimals', 'Default', 'Description', 'ID', 'Key',
       'MapYear', 'ParentID', 'Position', 'PresentationType', 'Title', 'Type',
       'Unit', 'odata.type'],
      dtype='object')


In [23]:
test_data.head(20)

,Datatype,Decimals,Default,Description,ID,Key,MapYear,ParentID,Position,PresentationType,Title,Type,Unit,odata.type
0,NaN,NaN,NaN,,0,WijkenEnBuurten,2019.0,NaN,0.0,NaN,Wijken en buurten,GeoDetail,NaN,Cbs.OData.GeoDetail
1,NaN,NaN,NaN,De gemeenten in Nederland zijn onderverdeeld i...,1,,NaN,NaN,NaN,NaN,Regioaanduiding,TopicGroup,NaN,Cbs.OData.TopicGroup
2,String,0.0,Impossible,De naam van de bestuurlijke gemeente. Deze naa...,2,Gemeentenaam_1,NaN,1.0,1.0,,Gemeentenaam,Topic,naam,Cbs.OData.Topic
3,String,0.0,Impossible,"De gekozen regioaanduiding betreft: Gemeente, ...",3,SoortRegio_2,NaN,1.0,2.0,,Soort regio,Topic,omschrijving,Cbs.OData.Topic
4,String,0.0,Impossible,"Gemeentecode heeft 4 posities, voorafgegaan do...",4,Codering_3,NaN,1.0,3.0,,Codering,Topic,code,Cbs.OData.Topic
5,String,0.0,Impossible,Deze indicator geeft per wijk en buurt aan of ...,5,IndelingswijzigingWijkenEnBuurten_4,NaN,1.0,4.0,,Indelingswijziging wijken en buurten,Topic,code,Cbs.OData.Topic
6,NaN,NaN,NaN,De bevolking van Nederland op 1 januari.\r\n\r...,6,,NaN,NaN,NaN,NaN,Bevolking,TopicGroup,NaN,Cbs.OData.TopicGroup
7,Long,0.0,Impossible,None,7,AantalInwoners_5,NaN,6.0,5.0,Absolute,Aantal inwoners,Topic,aantal,Cbs.OData.Topic
8,NaN,NaN,NaN,None,8,,NaN,6.0,NaN,NaN,Geslacht,TopicGroup,NaN,Cbs.OData.TopicGroup
9,Long,0.0,Impossible,None,9,Mannen_6,NaN,8.0,6.0,Absolute,Mannen,Topic,aantal,Cbs.OData.Topic
